Batch and Serving Layer with Redshift
===

Create a single-node Amazon Redshift data warehouse from the the admin EC2 linux instance

Before creating the Redshift cluster, let's prepare the Environment Variables **VPCSGID** and **SUBNET** with the following commands:

In [1]:
import boto3,os
cf = boto3.client('cloudformation', region_name='eu-west-1')
response = cf.describe_stacks(StackName=os.getenv('STACKNAME'))
outputs = response['Stacks'][0]['Outputs']
for output in outputs:
    if output['OutputKey'] == 'VPCSGID':
        os.environ['VPCSGID']=output['OutputValue']
    if output['OutputKey'] == 'Subnet':
        os.environ['SUBNET']=output['OutputValue']

In [2]:
!aws redshift create-cluster-subnet-group --cluster-subnet-group-name $USER$WORKSHOP --description $USER$WORKSHOP \
    --subnet-ids $SUBNET --region eu-west-1

{
    "ClusterSubnetGroup": {
        "Subnets": [
            {
                "SubnetStatus": "Active", 
                "SubnetIdentifier": "subnet-0f774bb671f63cd6a", 
                "SubnetAvailabilityZone": {
                    "Name": "eu-west-1a"
                }
            }
        ], 
        "VpcId": "vpc-07c0f1ff6c834c706", 
        "Description": "user6161018", 
        "Tags": [], 
        "SubnetGroupStatus": "Complete", 
        "ClusterSubnetGroupName": "user6161018"
    }
}


In [3]:
!aws redshift create-cluster --cluster-type single-node --vpc-security-group-ids $VPCSGID \
--node-type dc1.large --master-username admin --master-user-password Wpwd1216 \
--cluster-identifier $USER-$WORKSHOP-workshopcluster \
--publicly-accessible --region eu-west-1 --cluster-subnet-group-name $USER$WORKSHOP

{
    "Cluster": {
        "Encrypted": false, 
        "ClusterVersion": "1.0", 
        "NumberOfNodes": 1, 
        "VpcId": "vpc-07c0f1ff6c834c706", 
        "NodeType": "dc1.large", 
        "PubliclyAccessible": true, 
        "Tags": [], 
        "MasterUsername": "admin", 
        "ClusterParameterGroups": [
            {
                "ParameterGroupName": "default.redshift-1.0", 
                "ParameterApplyStatus": "in-sync"
            }
        ], 
        "AutomatedSnapshotRetentionPeriod": 1, 
        "EnhancedVpcRouting": false, 
        "IamRoles": [], 
        "AllowVersionUpgrade": true, 
        "VpcSecurityGroups": [
            {
                "Status": "active", 
                "VpcSecurityGroupId": "sg-0490f59a9fdd9c28c"
            }
        ], 
        "MaintenanceTrackName": "current", 
        "ClusterSubnetGroupName": "user6161018", 
        "ClusterSecurityGroups": [], 
        "ClusterStatus": "creating", 
        "ClusterIdentifier": "user6-16101

Time to create two policies and a new IAM role **dedicated to Redshift data import**

In [4]:
%%bash
echo """{
  \"Version\":\"2012-10-17\",
  \"Statement\": [
    {
      \"Effect\": \"Allow\",
      \"Principal\": {
        \"Service\": \"redshift.amazonaws.com\"
      },
      \"Action\": \"sts:AssumeRole\"
    }
  ]
}">assumerole-redshift-policy.json

In [5]:
%%bash
echo "{
  \"Version\": \"2012-10-17\",
  \"Statement\": [
    {
      \"Effect\": \"Allow\",
      \"Action\": [
        \"s3:Get*\",
        \"s3:List*\"
      ],
      \"Resource\": \"*\"
    }
  ]
}">s3-read-policy.json

In [6]:
!aws iam create-role --role-name $USER-$WORKSHOP-role-redshift-s3 \
  --assume-role-policy-document file://assumerole-redshift-policy.json

{
    "Role": {
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17", 
            "Statement": [
                {
                    "Action": "sts:AssumeRole", 
                    "Effect": "Allow", 
                    "Principal": {
                        "Service": "redshift.amazonaws.com"
                    }
                }
            ]
        }, 
        "RoleId": "AROAJVD27YA2EU6MJDOJO", 
        "CreateDate": "2018-10-16T09:09:59Z", 
        "RoleName": "user6-161018-role-redshift-s3", 
        "Path": "/", 
        "Arn": "arn:aws:iam::129182954833:role/user6-161018-role-redshift-s3"
    }
}


In [7]:
!aws iam put-role-policy --role-name $USER-$WORKSHOP-role-redshift-s3 --policy-name \
    $USER-$WORKSHOP-redshift-s3-r --policy-document file://s3-read-policy.json

You will have to **wait** your cluster on https://eu-west-1.console.aws.amazon.com/redshift/home?region=eu-west-1#cluster-list to be in **"Available" mode**.

When available it's time to attach the newly created IAM role to your cluster.

If you get a "Cluster is not in available state" response, please wait a little and retry.

In [17]:
!aws redshift modify-cluster-iam-roles --cluster-identifier "$USER-$WORKSHOP-workshopcluster" \
--add-iam-roles "arn:aws:iam::$ACCOUNTID:role/$USER-$WORKSHOP-role-redshift-s3" --region eu-west-1

{
    "Cluster": {
        "PubliclyAccessible": true, 
        "MasterUsername": "admin", 
        "VpcSecurityGroups": [
            {
                "Status": "active", 
                "VpcSecurityGroupId": "sg-0490f59a9fdd9c28c"
            }
        ], 
        "NumberOfNodes": 1, 
        "PendingModifiedValues": {}, 
        "VpcId": "vpc-07c0f1ff6c834c706", 
        "ClusterVersion": "1.0", 
        "Tags": [], 
        "AutomatedSnapshotRetentionPeriod": 1, 
        "ClusterParameterGroups": [
            {
                "ParameterGroupName": "default.redshift-1.0", 
                "ParameterApplyStatus": "in-sync"
            }
        ], 
        "PreferredMaintenanceWindow": "thu:00:00-thu:00:30", 
        "Endpoint": {
            "Port": 5439, 
            "Address": "user6-161018-workshopcluster.cm633nfk9ggq.eu-west-1.redshift.amazonaws.com"
        }, 
        "IamRoles": [
            {
                "ApplyStatus": "adding", 
                "IamRoleArn": "arn:a

See the following command to check the state.

In [9]:
!aws redshift describe-clusters --cluster-identifier $USER-$WORKSHOP-workshopcluster --region eu-west-1

{
    "Clusters": [
        {
            "PubliclyAccessible": true, 
            "MasterUsername": "admin", 
            "VpcSecurityGroups": [
                {
                    "Status": "active", 
                    "VpcSecurityGroupId": "sg-0490f59a9fdd9c28c"
                }
            ], 
            "ClusterPublicKey": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCe4Lgb66VaZF+kqadZp50Tm2QkOErYo205IHrr7l/Jpr7A39JA8TmC7GtmYUxcykX578Sy7NQM7+ZiiIqZuLmUuWOPdj9S3EjncficPrlcicPixfDuRN1KzIrmEJMAcCYKeVuGkDqUVFyouibYopm3anrPR4uxCtlncoyYzikJMdFtxW3+dkUNAHvV0EiUbOqNY+GW15mKYewdUB/tNUfs70v1RFz8m7lAlqOp/WOFP1c0PkyRG0cEgdBykFT2ck9uUH+5N2EQa1+5f6JvRADg2/+hLprTZWdrqdYSZOBt4/hghOUcI1i71nVjlSkI/hkiM8AJw6yJrl53AAyOXGNf Amazon-Redshift\n", 
            "NumberOfNodes": 1, 
            "PendingModifiedValues": {}, 
            "VpcId": "vpc-07c0f1ff6c834c706", 
            "ClusterVersion": "1.0", 
            "Tags": [], 
            "AutomatedSnapshotRetentionPeriod": 1, 
            "ClusterPara

**Now it's time to use our database**

For that time
 - 1st to create a table
 - 2nd to test an insertion
 - 3rd to use COPY to load a sample of 5 millions lines
 
**don't forget to update REDSHIFT-URL with URL coming from previous describe-cluster**

In [10]:
env REDSHIFTURL=user6-161018-workshopcluster.cm633nfk9ggq.eu-west-1.redshift.amazonaws.com

env: REDSHIFTURL=user6-161018-workshopcluster.cm633nfk9ggq.eu-west-1.redshift.amazonaws.com


In [11]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 dev -U admin \
 -c 'create table hillary (id bigint, name varchar, text varchar(1024),time bigint,isodate timestamp);'

CREATE TABLE


In [12]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 dev -U admin \
 -c "select * from pg_table_def where tablename = 'hillary';"

 schemaname | tablename | column  |            type             | encoding | distkey | sortkey | notnull 
------------+-----------+---------+-----------------------------+----------+---------+---------+---------
 public     | hillary   | id      | bigint                      | lzo      | f       |       0 | f
 public     | hillary   | name    | character varying(256)      | lzo      | f       |       0 | f
 public     | hillary   | text    | character varying(1024)     | lzo      | f       |       0 | f
 public     | hillary   | time    | bigint                      | lzo      | f       |       0 | f
 public     | hillary   | isodate | timestamp without time zone | lzo      | f       |       0 | f
(5 rows)



LZO encoding provides a very high compression ratio with good performance. LZO encoding works especially well for CHAR and VARCHAR columns that store very long character strings, especially free form text, such as product descriptions, user comments, or JSON strings. LZO is the default encoding except for columns that are designated as sort keys and columns that are defined as BOOLEAN, REAL, or DOUBLE PRECISION data types.

In [13]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c 'select * from hillary;'

 id | name | text | time | isodate 
----+------+------+------+---------
(0 rows)



In [14]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c \
    "insert into hillary values (777263619033661400,'User','Test message',1474418015000,'2016-09-17 21:51:02');"

INSERT 0 1


In [15]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c 'select * from hillary;'

         id         | name |     text     |     time      |       isodate       
--------------------+------+--------------+---------------+---------------------
 777263619033661400 | User | Test message | 1474418015000 | 2016-09-17 21:51:02
(1 row)



In [19]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL \
  -p 5439 dev -U admin \
    -c "copy hillary from 's3://$USER-$WORKSHOP-aws-bigdata-workshop/sample/Hillary-aa' escape region 'eu-west-1' \
      credentials 'aws_iam_role=arn:aws:iam::$ACCOUNTID:role/$USER-$WORKSHOP-role-redshift-s3' dateformat 'auto';"

INFO:  Load into table 'hillary' completed, 5000000 record(s) loaded successfully.
COPY


By default, the COPY command applies automatic compression whenever you run the COPY command with an empty target table and all of the table columns either have RAW encoding or no encoding.

In [20]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 dev -U admin \
 -c "analyze compression hillary;"

  Table  | Column  | Encoding | Est_reduction_pct 
---------+---------+----------+-------------------
 hillary | id      | zstd     | 12.89
 hillary | name    | zstd     | 30.16
 hillary | text    | zstd     | 28.80
 hillary | time    | zstd     | 52.53
 hillary | isodate | zstd     | 45.22
(5 rows)



ANALYZE COMPRESSION performs compression analysis and produces a report with the suggested compression encoding for the tables analyzed. For each column, the report includes an estimate of the potential reduction in disk space compared to the current encoding.

Now we have the data, time to play with Redshift

In [21]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c 'select id,name,substring(text,0,32) as message,isodate from hillary limit 10;'

         id         |      name       |               message               |       isodate       
--------------------+-----------------+-------------------------------------+---------------------
 780543483404808200 | williamjones8   | RT @HillaryClinton: ���You didn���t | 2016-09-26 23:04:02
 780543482993582100 | hassaaninator   | RT @HillaryClinton: ���You didn���t | 2016-09-26 23:04:02
 780543482024763400 | GabrielleLozano | RT @Jeritron2016: @HillaryClint     | 2016-09-26 23:04:02
 780543480674345000 | FrankDiFlorio1  | RT @HillaryClinton: ���You didn���t | 2016-09-26 23:04:02
 780543479411830800 | JennDub2        | RT @HillaryClinton: ���You didn���t | 2016-09-26 23:04:01
 780543479160070100 | MikeRGreenhalgh | RT @FoxNews: Poll: @realDonaldT     | 2016-09-26 23:04:01
 780543478656671700 | DamnGoodPolls   | Are you going to watch the pres     | 2016-09-26 23:04:01
 780543477562081300 | SebastianRio    | I'm excited about #debatenight      | 2016-09-26 23:04:01
 78054347653456690

In [22]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c 'select count(*) from hillary;'

  count  
---------
 5000001
(1 row)



In [23]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c 'select count(distinct id) from hillary;'

  count  
---------
 4211397
(1 row)



In [24]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c 'select count(distinct name) from hillary;'

  count  
---------
 1184930
(1 row)



**Given the fact we have duplicates, we will create a new table with deduplicated data**

In [25]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c 'create table hillarydeduplicated \
  (id bigint encode zstd, name varchar encode zstd, text varchar(1024) encode zstd,time bigint encode zstd, \
      isodate timestamp encode zstd) distkey(isodate) ;'

CREATE TABLE


In [26]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 dev -U admin \
 -c "select * from pg_table_def where tablename = 'hillarydeduplicated';"

 schemaname |      tablename      | column  |            type             | encoding | distkey | sortkey | notnull 
------------+---------------------+---------+-----------------------------+----------+---------+---------+---------
 public     | hillarydeduplicated | id      | bigint                      | zstd     | f       |       0 | f
 public     | hillarydeduplicated | name    | character varying(256)      | zstd     | f       |       0 | f
 public     | hillarydeduplicated | text    | character varying(1024)     | zstd     | f       |       0 | f
 public     | hillarydeduplicated | time    | bigint                      | zstd     | f       |       0 | f
 public     | hillarydeduplicated | isodate | timestamp without time zone | zstd     | t       |       0 | f
(5 rows)



In [27]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c 'insert into hillarydeduplicated (select distinct * from hillary);'

INSERT 0 4212150


**Now we can start some analysis on our sample**

In [28]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c "select count(*) from hillarydeduplicated where text ilike '%bill%clinton%';"

 count 
-------
 90107
(1 row)



In [29]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c "select count(*) from hillarydeduplicated where text ilike '%obama%';"

 count  
--------
 147599
(1 row)



LIKE is case-sensitive; ILIKE is case-insensitive.

Now, time to play with data. 

In [30]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439\
  dev -U admin -c "truncate table hillary;"

TRUNCATE TABLE and COMMIT TRANSACTION


In [31]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439\
  dev -U admin -c "truncate table hillarydeduplicated;"

TRUNCATE TABLE and COMMIT TRANSACTION


Deletes all of the rows from a table without doing a table scan: this operation is a faster alternative to an unqualified DELETE operation. To execute a TRUNCATE command, you must be the owner of the table or a superuser.

TRUNCATE is much more efficient than DELETE and does not require a VACUUM and ANALYZE. However, be aware that TRUNCATE commits the transaction in which it is run.

In [32]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL \
  -p 5439 dev -U admin \
    -c "copy hillary from 's3://$USER-$WORKSHOP-aws-bigdata-workshop/split/Hillary/' escape region 'eu-west-1' \
      credentials 'aws_iam_role=arn:aws:iam::$ACCOUNTID:role/$USER-$WORKSHOP-role-redshift-s3' dateformat 'auto';"

INFO:  Load into table 'hillary' completed, 27808777 record(s) loaded successfully.
COPY


In [33]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c 'insert into hillarydeduplicated (select distinct * from hillary);'

INSERT 0 17115623


In [34]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439\
  dev -U admin -c "vacuum;analyze;"

ANALYZE SKIP


VACUUM : Reclaims space and resorts rows in either a specified table or all tables in the current database.

ANALYZE : Updates table statistics for use by the query planner.

In [35]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c "select count(*) from hillarydeduplicated where text ilike '%obama%';"

 count  
--------
 535432
(1 row)



REDSHIFT SPECTRUM and REDSHIFT (EU-WEST-1)
===

Amazon Redshift Spectrum requires engine version 1.0.1294 or later. To find the version number for your cluster, run the following command.

In [36]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 dev -U admin -c "select version();"

                                                         version                                                          
--------------------------------------------------------------------------------------------------------------------------
 PostgreSQL 8.0.2 on i686-pc-linux-gnu, compiled by GCC gcc (GCC) 3.4.2 20041017 (Red Hat 3.4.2-6.fc3), Redshift 1.0.4349
(1 row)



To create an external schema, replace the IAM role ARN in the following command with the role ARN you created in step 1, and then execute the command in your SQL client. 

In [37]:
%%bash
echo "{
        \"Version\": \"2012-10-17\",
        \"Statement\": [
            {
                \"Effect\": \"Allow\",
                \"Action\": [\"athena:*\"],
                \"Resource\": [\"*\"]
            },
            {
                \"Effect\": \"Allow\",
                \"Action\": [\"glue:*\"],
                \"Resource\": [\"*\"]
            }
        ]
}">redshift-spectrum-policy.json

In [38]:
!aws iam put-role-policy --role-name $USER-$WORKSHOP-role-redshift-s3 --policy-name \
    $USER-$WORKSHOP-redshift-spectrum --policy-document file://redshift-spectrum-policy.json

In [40]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439\
  dev -U admin -c "create external schema potus_spectrum_$USER$WORKSHOP \
  from data catalog database 'potus_spectrum_db_$USER$WORKSHOP' \
  iam_role 'arn:aws:iam::$ACCOUNTID:role/$USER-$WORKSHOP-role-redshift-s3' \
  create external database if not exists;"

INFO:  External database "potus_spectrum_db_user6161018" created
CREATE SCHEMA


Some applications use the term database and schema interchangeably. In Amazon Redshift, we use the term schema.

The previous example creates an external schema named potus_spectrum using the external database potus_spectrum_db.

In [41]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439\
  dev -U admin -c "create external table \
  potus_spectrum_$USER$WORKSHOP.clinton(id bigint,name varchar,message varchar(1024),ts bigint, \
  isodate timestamp, date varchar) row format delimited fields terminated by '|' stored as textfile \
  location 's3://aws-potus-eu-west-1/split/Hillary'"

CREATE EXTERNAL TABLE


In [42]:
!export PGPASSWORD='Wpwd1216';psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c "select id,name,substring(message,0,32) as message,isodate \
      from potus_spectrum_$USER$WORKSHOP.clinton limit 10;"

         id         |      name       |               message               |       isodate       
--------------------+-----------------+-------------------------------------+---------------------
 780543483404808200 | williamjones8   | RT @HillaryClinton: ���You didn���t | 2016-09-26 23:04:02
 780543483258044400 | MWChin          | Starting with: Are you a perjur     | 2016-09-26 23:04:02
 780543482993582100 | hassaaninator   | RT @HillaryClinton: ���You didn���t | 2016-09-26 23:04:02
 780543482486284300 | TJ_Pittinger    | One of y'all let the 2013 @FSU_     | 2016-09-26 23:04:02
 780543482024763400 | GabrielleLozano | RT @Jeritron2016: @HillaryClint     | 2016-09-26 23:04:02
 780543481349537800 | Anomaly100      | RT @HillaryClinton: Watch this      | 2016-09-26 23:04:02
 780543480674345000 | FrankDiFlorio1  | RT @HillaryClinton: ���You didn���t | 2016-09-26 23:04:02
 780543480489586700 | All4Hillary     | RT @HillaryIn2016: Who's ready      | 2016-09-26 23:04:02
 78054347941183080

In [43]:
!export PGPASSWORD='Wpwd1216';time -p psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c "select count(distinct id) from potus_spectrum_$USER$WORKSHOP.clinton;"

  count   
----------
 17111780
(1 row)

real 16.44
user 0.00
sys 0.00


In [44]:
!export PGPASSWORD='Wpwd1216';time -p psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c 'select count(distinct id) from hillary;'

  count   
----------
 17111780
(1 row)

real 5.66
user 0.00
sys 0.00


In [45]:
!export PGPASSWORD='Wpwd1216';time -p psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c "select count(*) from potus_spectrum_$USER$WORKSHOP.clinton where lower(message) like '%obama%';"

  count  
---------
 1045581
(1 row)

real 11.43
user 0.00
sys 0.00


In [46]:
!export PGPASSWORD='Wpwd1216';time -p psql -h $REDSHIFTURL -p 5439 \
  dev -U admin -c "select count(*) from hillary where text ilike '%obama%';"

  count  
---------
 1045745
(1 row)

real 16.01
user 0.00
sys 0.00


In Presto like is case sensitive => we use the workaround based on lower(message) to have a case insensitive query.

Redshift ilike is case insensitive.